[View in Colaboratory](https://colab.research.google.com/github/meghamattikalli/lstm/blob/master/lstm_95_20_epochs.ipynb)

In [1]:
import os

from google.colab import files
uploaded = files.upload()

Saving five_thousand.csv to five_thousand.csv
Saving Indian-Female-Names1.csv to Indian-Female-Names1.csv
Saving Indian-Male-Names1.csv to Indian-Male-Names1.csv


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 29 00:07:33 2018
acc= 95.52
20 epochs

@author: HP
"""


from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np


import io

female = pd.read_csv(io.StringIO(uploaded['Indian-Female-Names1.csv'].decode('utf-8')))

male = pd.read_csv(io.StringIO(uploaded['Indian-Male-Names1.csv'].decode('utf-8')))

eng_word = pd.read_csv(io.StringIO(uploaded['five_thousand.csv'].decode('utf-8')))


#parameters
maxlen = 10
labels = 2

# Importing the dataset
#female= pd.read_csv('Indian-Female-Names1.csv')
#male= pd.read_csv('Indian-Male-Names1.csv')
names=pd.concat([female,male], axis=0)
names= names.apply(lambda x: x.astype(str).str.lower())
names=names.iloc[:,0:1]
names["type"]=1
names=names[names['word'].map(len) > 2]

unwanted=['mr','.','moh.','@','miss','&','mrs','/','ku.','km',',','km0','-','(',')','smt','smt.','`','[','ms','1','2','3','4','5','6','7','8','9','0']
for i in unwanted:
    names['word'] = names.word.str.replace(i, '')
names['word'] = names.word.str.strip()
names=names.drop_duplicates(subset=['word'], keep='first')
names['word'] = names.word.str.partition(" ")


#english words
#eng_word = pd.read_csv('five_thousand.csv')
eng_word = eng_word.apply(lambda x: x.astype(str).str.lower())
eng_word['word'] = eng_word.word.str.replace('-', '')
eng_word['word'] = eng_word.word.str.replace("'", '')
eng_word['word'] = eng_word.word.str.replace("/", ' ')
eng_word['word'] = eng_word.word.str.partition(" ")


#eng_pre = pd.read_csv('prepositions.csv')
eng_word['type'] = 0
#eng_pre['type'] = 0

#concatenate names and english words
dataset=pd.concat([names,eng_word], axis=0)
dataset=dataset.dropna()

dataset.index=[i for i in range(0,len(dataset.index))] #renaming the indices
features = dataset['word']
label = dataset['type']

vocab = set(' '.join([str(i) for i in features]))

vocab.add('END')
len_vocab = len(vocab)

print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(dataset))

char_index = dict((c, i) for i, c in enumerate(vocab))

print(char_index)


#train test split
msk = np.random.rand(len(dataset)) < 0.8
train = dataset[msk]
test = dataset[~msk]


def set_flag(i):
    tmp = np.zeros(28);
    tmp[i] = 1
    return(tmp)
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.word]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.type:
    if i == 0:
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])

np.asarray(train_X).shape

np.asarray(train_Y).shape

"""#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)"""

#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.word]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.type:
    if i == 0:
        test_Y.append([1,0]) 
    else:
        test_Y.append([0,1])
        

print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

train_X=np.array(train_X)
train_Y=np.array(train_Y)
test_Y=np.array(test_Y)
test_X=np.array(test_X)


batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=20,validation_data=(test_X, test_Y))

score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)




Using TensorFlow backend.


{'u', 'c', 'r', ' ', 'z', 'j', 's', 'd', 'o', 'g', 'i', 'v', 't', 'END', 'n', 'q', 'a', 'f', 'h', 'l', 'p', 'e', 'm', 'y', 'k', 'b', 'w', 'x'}
vocab length is  28
length of input is  18716
{'u': 0, 'c': 1, 'r': 2, ' ': 3, 'z': 4, 'j': 5, 's': 6, 'd': 7, 'o': 8, 'g': 9, 'i': 10, 'v': 11, 't': 12, 'END': 13, 'n': 14, 'q': 15, 'a': 16, 'f': 17, 'h': 18, 'l': 19, 'p': 20, 'e': 21, 'm': 22, 'y': 23, 'k': 24, 'b': 25, 'w': 26, 'x': 27}
Build model...
(3750, 10, 28)
(3750, 2)


/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 14966 samples, validate on 3750 samples
Epoch 1/50
14966/14966 [==============================] - 78s 5ms/step - loss: 0.4910 - acc: 0.7420 - val_loss: 0.3626 - val_acc: 0.8437
Epoch 2/50
14966/14966 [==============================] - 74s 5ms/step - loss: 0.3493 - acc: 0.8480 - val_loss: 0.3244 - val_acc: 0.8576
Epoch 3/50
14966/14966 [==============================] - 75s 5ms/step - loss: 0.3211 - acc: 0.8598 - val_loss: 0.2961 - val_acc: 0.8752
Epoch 4/50
14966/14966 [==============================] - 75s 5ms/step - loss: 0.3036 - acc: 0.8700 - val_loss: 0.2757 - val_acc: 0.8907
Epoch 5/50
14966/14966 [==============================] - 76s 5ms/step - loss: 0.2835 - acc: 0.8818 - val_loss: 0.2617 - val_acc: 0.8952
Epoch 6/50
14966/14966 [==============================] - 74s 5ms/step - loss: 0.2724 - acc: 0.8887 - val_loss: 0.2493 - val_acc: 0.9016
Epoch 7/50
14966/14966 [==============================] - 74s 5ms/step - loss: 0.2613 - acc: 0.8946 - val_loss: 0.2554 - val_acc:

In [1]:
train_X

NameError: ignored